<a href="https://colab.research.google.com/github/dev-SB/cricket-bi/blob/master/bi_cricket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import pandas as pd
import os

In [0]:
# files=[file for dirpath,directory,file in os.walk('./all_csv/')][0]
files=[file for dirpath,directory,file in os.walk(r'/gdrive/My Drive/all_csv/')][0]

In [0]:
match_data=pd.DataFrame(data=None)
odi_scorecard=pd.DataFrame(data=None)
ttwenty_scorecard=pd.DataFrame(data=None)
odi_info=pd.DataFrame(data=None)
ttwenty_info=pd.DataFrame(data=None)

In [0]:
def rename_date_umpire(index_list):
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='date':
            index_list[i]+='_'+str(n)
            n+=1
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='umpire':
            index_list[i]+='_'+str(n)
            n+=1
    return index_list

In [0]:
def find_game(df_game,df_info):
    if 'series' in df_info.columns:
        if 'odi' in str.lower(df_info.iloc[0]['series']):
            return 'odi'
        if 't20i' in str.lower(df_info.iloc[0]['series']):
            return 'twenty'
    if max(df_game['balls_bowl'])<=24:
        return 'twenty'
    if 24<max(df_game['balls_bowl'])<=60:
        return 'odi'
    return

In [0]:
def append_file(temp_df,temp_info_df,type_game):
    global odi_scorecard
    global ttwenty_scorecard
    global odi_info
    global ttwenty_info
    if type_game=='odi':
        odi_scorecard=odi_scorecard.append(temp_df,ignore_index=True)
        odi_info=odi_info.append(temp_info_df,ignore_index=True)
    elif type_game=='twenty':
        ttwenty_scorecard=ttwenty_scorecard.append(temp_df,ignore_index=True)
        ttwenty_info=ttwenty_info.append(temp_info_df,ignore_index=True)

In [0]:
def get_extras_type(match_data):
    list_extras=[]
    for index,row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
          ov=str(row['over']).split('.')
          ball_no=int(ov[1])
          over_no=int(ov[0])
        else:
          continue
        if row['extras']!=0:
            if (row['extras']==1 or row['extras']==3) and index+1<match_data.shape[0] and (match_data.loc[index+1,'striker']==row['non-striker'] or match_data.loc[index+1,'non-striker']==row['striker']):
                match_data.loc[index,'extras_type']='b'
            else:
                list_extras.append(index)
        if ball_no>6:
            if len(list_extras)>0:
                match_data.loc[list_extras.pop(-1),'extras_type']='w'
    for i in list_extras:
        match_data.loc[i,'extras_type']='b'
    return match_data

In [0]:
def prepare_scorecard(match_data):
    match_data=get_extras_type(match_data)
#     print(match_data[match_data['bowler']==	'Mashrafe Mortaza'])
    players=list((match_data['striker'].append(match_data['non-striker']).append(match_data['bowler'])).unique())
    #     to make 22 players if any player has not played
#     for i in range(len(players),22):
#         players.append('p_'+str(i))
#         fow
#     player_stats=['match_no','bats_innings','name','position','over_batting','score','balls_bat','ones','twos','threes','fours','sixes','out','balls_bowl','maidens','runs_bowled','wickets','bowler_extras','wicket_score','wicket_overs','wicket_no','fow','fow_runs','fow_overs','fow_batsman','fow_bowler']
    player_stats=['match_no','bats_innings','name','position','over_batting','score','balls_bat','ones','twos','threes','fours','sixes','out','balls_bowl','maidens','runs_bowled','wickets','bowler_extras','wicket_score','wicket_overs','wicket_no']
    player_data={key:{key_type:0 for key_type in player_stats} for key in players}
    for p in players:
        player_data[p]['match_no']=match_data.loc[0,'file_no']
        player_data[p]['name']=p
    team_score=0
    balls=0
    pos=1
    inning=False
    w=1
    p_no=1
    w_no=1
    for index, row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
          ov=ov.split('.')
          ball_no=int(ov[1])
          over_no=int(ov[0])
        else:
          continue
        if over_no>50:
          player_data=[value for key, value in player_data.items()]
          scorecard=pd.DataFrame(data=player_data)
          scorecard=scorecard[player_stats]
          return scorecard
        if ball_no==1:
            pos=1
            w=1
            team_score=0
            w_no=1
            runs_over=0
            # if row['innings']!=1:
            #     p_no=12
        if row['runs']==1:
            player_data[row['striker']]['ones']+=1
        elif row['runs']==2:
            player_data[row['striker']]['twos']+=1
        elif row['runs']==3:
            player_data[row['striker']]['threes']+=1
        elif row['runs']==4:
            player_data[row['striker']]['fours']+=1
        elif row['runs']==6:
            player_data[row['striker']]['sixes']+=1

        if player_data[row['striker']]['position']==0:
            player_data[row['striker']]['position']=pos
            player_data[row['striker']]['over_batting']=row['over']
            pos+=1
        if player_data[row['non-striker']]['position']==0:
            player_data[row['non-striker']]['position']=pos
            player_data[row['non-striker']]['over_batting']=row['over']
            pos+=1
    # wicket
#         print(row['out-player'])
        if not pd.isna(row['out-player']):
            player_data[row['out-player']]['out']=True
#             fow
#             player_data[players[p_no-1]]['fow']=w
#             player_data[players[p_no-1]]['fow_runs']=team_score
#             player_data[players[p_no-1]]['fow_overs']=row['over']
#             player_data[players[p_no-1]]['fow_batsman']=row['out-player']
#             player_data[players[p_no-1]]['fow_bowler']=row['bowler']
            p_no+=1
            w+=1
            if row['out']!='run out':
                player_data[row['bowler']]['wickets']+=1
            player_data[row['out-player']]['wicket_score']=team_score
            player_data[row['out-player']]['wicket_overs']=row['over']
            player_data[row['out-player']]['wicket_no']=w_no
            w_no+=1

        team_score+=row['runs']+row['extras']
        runs_over+=row['runs']
        if row['extras']!=0 and row['extras_type']=='w':
            runs_over+=1
            player_data[row['bowler']]['bowler_extras']+=1
            player_data[row['striker']]['score']+=row['extras']-1
            player_data[row['bowler']]['runs_bowled']+=row['extras']-1
        elif row['extras']!=0:
#             print(row)
            player_data[row['bowler']]['balls_bowl']+=1
            player_data[row['striker']]['balls_bat']+=1
        else:
            player_data[row['striker']]['score']+=row['runs']
            player_data[row['striker']]['balls_bat']+=1
            player_data[row['bowler']]['balls_bowl']+=1
            player_data[row['bowler']]['runs_bowled']+=row['runs']
        if ball_no>=6:
            if ball_no==6 and runs_over==0:
                player_data[row['bowler']]['maidens']+=1
            runs_over=0
        player_data[row['striker']]['bats_innings']=row['innings']
        if row['innings']==1:
            player_data[row['bowler']]['bats_innings']=2
        else:
            player_data[row['bowler']]['bats_innings']=1
#     print(player_data)
    player_data=[value for key, value in player_data.items()]
    scorecard=pd.DataFrame(data=player_data)
    scorecard=scorecard[player_stats]
    return scorecard

In [0]:
i=0
for file in files[:]:
    print(i,'--',file)
    i+=1
    count=0
    df_index=[]
    df_row=[]
    # add=r'./all_csv/'+file
    add=r'/gdrive/My Drive/all_csv/'+file
    df_index=['file_no']
    file_no=file.split('.')[0]
    df_row=[file_no]
    with open(add) as f:
        new_f=f.readlines()
        for line in new_f:
            if 'version' in line:
                count+=1
            elif 'info' in line:
                count+=1
                line=line.strip().split(',')
                df_index.append(line[1])
                df_row.append(line[2])
            else:
                df_index=rename_date_umpire(df_index)
                df_dic=dict(zip(df_index,df_row),index=[0])
                temp_info_df=pd.DataFrame(df_dic)
#                 df_info=df_info.append(temp_info_df,ignore_index=True)
                # gender=df_info['gender'].iloc[0]
                # gender=str.lower(gender.strip())
                break
    temp_df=pd.read_csv(add,skiprows=count,names=[0,'innings','over','batting-team','striker','non-striker','bowler','runs','extras','out','out-player'])
    temp_df=temp_df.drop([0],axis=1)
    temp_df['file_no']=[file_no]*(temp_df.shape[0])
    temp_sc=prepare_scorecard(temp_df)
    # print(temp_sc)
#     append_file(temp_df,gender,type_game) 
    append_file(temp_sc,temp_info_df,find_game(temp_sc,temp_info_df))

0 -- 426414.csv
1 -- 654093.csv
2 -- 256608.csv
3 -- 829761.csv
4 -- 1031437.csv


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


5 -- 571247.csv
6 -- 533297.csv
7 -- 548376.csv
8 -- 656465.csv
9 -- 433568.csv
10 -- 548320.csv
11 -- 754719.csv
12 -- 603244.csv
13 -- 682947.csv
14 -- 635658.csv
15 -- 995455.csv
16 -- 426415.csv
17 -- 1030215.csv
18 -- 423783.csv
19 -- 533280.csv
20 -- 1114885.csv
21 -- 439144.csv
22 -- 446956.csv
23 -- 875509.csv
24 -- 336039.csv
25 -- 829717.csv
26 -- 656443.csv
27 -- 483126.csv
28 -- 1023585.csv
29 -- 467092.csv
30 -- 598027.csv
31 -- 251488.csv
32 -- 1082608.csv
33 -- 406199.csv
34 -- 966761.csv
35 -- 980961.csv
36 -- 682993.csv
37 -- 635653.csv
38 -- 897757.csv
39 -- 643675.csv
40 -- 524930.csv
41 -- 433586.csv
42 -- 1175363.csv
43 -- 1030217.csv
44 -- 440946.csv
45 -- 644945.csv
46 -- 647251.csv
47 -- 800471.csv
48 -- 300443.csv
49 -- 980911.csv
50 -- 238199.csv
51 -- 247486.csv
52 -- 335358.csv
53 -- 433606.csv
54 -- 518948.csv
55 -- 439142.csv
56 -- 1085971.csv
57 -- 1136575.csv
58 -- 750671.csv
59 -- 1114894.csv
60 -- 1136572.csv
61 -- 567365.csv
62 -- 656491.csv
63 -- 598

In [0]:
odi_info.columns

In [0]:
ttwenty_info.columns

In [0]:
odi_scorecard

In [0]:
ttwenty_info

In [0]:
ttwenty_scorecard

In [0]:
odi_info.

In [0]:
odi_info.to_csv('/gdrive/My Drive/odi_info.csv',index=False)
ttwenty_info.to_csv('/gdrive/My Drive/ttwenty_info.csv',index=False)
odi_scorecard.to_csv('/gdrive/My Drive/odi_scorecard.csv',index=False)
ttwenty_scorecard.to_csv('/gdrive/My Drive/ttwenty_scorecard.csv',index=False)